# Bank Customer Churn Prediction

Use logistic regression to predict churn

* Dataset: https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset

## Data preparation

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('customer-churn-records.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [3]:
df.head().T

,0,1,2,3,4
customer_id,15634602,15647311,15619304,15701354,15737888
credit_score,619,608,502,699,850
country,france,spain,france,france,spain
gender,female,female,female,female,female
age,42,41,42,39,43
tenure,2,1,8,1,2
balance,0.0,83807.86,159660.8,0.0,125510.82
products_number,1,1,3,2,1
credit_card,1,0,1,0,1
active_member,1,1,0,0,1


## Setting up the validation framework

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

## EDA

In [5]:
df_full_train.isnull().sum()

customer_id         0
credit_score        0
country             0
gender              0
age                 0
tenure              0
balance             0
products_number     0
credit_card         0
active_member       0
estimated_salary    0
churn               0
dtype: int64

In [6]:
df_full_train.churn.value_counts(normalize=True)

churn
0    0.79725
1    0.20275
Name: proportion, dtype: float64

In [7]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

np.float64(0.2)

In [8]:
numerical = [
            'credit_score',
            'age',
            'tenure',
            'balance',
            'products_number',
            'credit_card',
            'active_member',
            'estimated_salary'
        ]

categorical = [
        'country',
        'gender'
    ]

In [9]:
df_full_train[numerical].nunique()

credit_score         456
age                   69
tenure                11
balance             5105
products_number        4
credit_card            2
active_member          2
estimated_salary    7999
dtype: int64

## Feature importance: Churn rate and risk ration

In [10]:
df_full_train.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
2694,15721585,628,germany,male,29,3,113146.98,2,0,1,124749.08,0
5140,15617790,626,france,female,29,4,105767.28,2,0,0,41104.82,0
2568,15775905,612,germany,female,47,6,130024.87,1,1,1,45750.21,1
3671,15616666,646,germany,female,52,6,111739.40,2,0,1,68367.18,0
7427,15664720,714,spain,male,33,8,122017.19,1,0,0,162515.17,0


In [11]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

np.float64(0.24931431705979154)

In [12]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

np.float64(0.16375746440055122)

In [13]:
global_churn = df_full_train.churn.mean()
global_churn

np.float64(0.20275)

In [14]:
global_churn - churn_female

np.float64(-0.04656431705979153)

In [15]:
global_churn - churn_male

np.float64(0.03899253559944879)

In [16]:
for c in categorical:
  print(c)
  df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
  df_group['diff'] = df_group['mean'] - global_churn
  df_group['risk'] = df_group['mean'] - global_churn
  display(df_group)
  print()
  print()

country


,mean,count,diff,risk
country,,,,
france,0.160991,3994,-0.041759,-0.041759
germany,0.318227,2030,0.115477,0.115477
spain,0.168522,1976,-0.034228,-0.034228




gender


,mean,count,diff,risk
gender,,,,
female,0.249314,3646,0.046564,0.046564
male,0.163757,4354,-0.038993,-0.038993


## Feature importance: Correlation

In [17]:
df_full_train[numerical].corrwith(df_full_train.churn)

credit_score       -0.026420
age                 0.278079
tenure             -0.008248
balance             0.116926
products_number    -0.040868
credit_card        -0.014177
active_member      -0.159633
estimated_salary    0.006483
dtype: float64

In [18]:
cr = df_full_train[numerical].corrwith(df_full_train.churn).abs()
cr.sort_values(ascending=False)

age                 0.278079
active_member       0.159633
balance             0.116926
products_number     0.040868
credit_score        0.026420
credit_card         0.014177
tenure              0.008248
estimated_salary    0.006483
dtype: float64

## Training Model

In [19]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000, solver='liblinear', class_weight='balanced')
    model.fit(X_train, y_train)
    
    return dv, model


In [20]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [21]:
C = 1.0
n_splits = 5

## Validation Model

In [22]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values


    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.737 +- 0.023


In [23]:
scores

[np.float64(0.7093175853018372),
 np.float64(0.7131032758189547),
 np.float64(0.7537632344262178),
 np.float64(0.7367352364073676),
 np.float64(0.7696151496074656)]

In [24]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

np.float64(0.7475732583330167)

In [25]:
import pickle


Save the model

In [26]:
output_file = f'model_C={C}.bin'

In [27]:
output_file

'model_C=1.0.bin'

In [28]:
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()

In [29]:
!ls -lh *.bin

-rw-r--r--  1 senalidilumika  staff   1.2K Nov 25 20:31 model_C=1.0.bin


In [30]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

Load the model

In [31]:
import pickle

In [32]:
input_file = 'model_C=1.0.bin'

In [33]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [34]:
model

LogisticRegression(class_weight='balanced', max_iter=1000, solver='liblinear')

In [35]:
customer = {
 'country': 'germany',
 'gender': 'male',
 'credit_score': 655,
 'age': 52,
 'tenure': 9,
 'balance': 144696.75,
 'products_number': 1,
 'credit_card': 1,
 'active_member': 1,
 'estimated_salary': 49025.79
 }

In [36]:
X = dv.transform([customer])

In [37]:
y_pred = model.predict_proba(X)[0, 1]

In [38]:
print('input:', customer)
print('output:', y_pred)

input: {'country': 'germany', 'gender': 'male', 'credit_score': 655, 'age': 52, 'tenure': 9, 'balance': 144696.75, 'products_number': 1, 'credit_card': 1, 'active_member': 1, 'estimated_salary': 49025.79}
output: 0.683151062071004


Making requests

In [39]:
import requests

In [40]:
url = 'http://localhost:9696/predict'

In [41]:
customer = {
 'country': 'germany',
 'gender': 'male',
 'credit_score': 655,
 'age': 52,
 'tenure': 9,
 'balance': 144696.75,
 'products_number': 1,
 'credit_card': 1,
 'active_member': 1,
 'estimated_salary': 49025.79
 }

In [42]:
response = requests.post(url, json=customer).json()
response

{'churn': True, 'churn_probability': 0.683151062071004}

In [43]:
if response['churn']:
    print('sending email to', 'asdx-123d')

sending email to asdx-123d
